In [7]:
import pandas as pd
import nltk
import re
import transformers
#from tensorflow.python.keras.models import transformers
import numpy as np
import matplotlib.pyplot as plt
import warnings
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, roc_curve
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from catboost import CatBoostClassifier
from nltk.corpus import stopwords as nltk_stopwords
from pymystem3 import Mystem
from tqdm.notebook import tqdm
from wordcloud import WordCloud
warnings.filterwarnings('ignore')

2023-06-21 18:03:59.503137: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [48]:
# Загружаем каждое предложение в отдельную строку df. Ставим сепоратор таб, иначе запятая выдает ошибку
df = pd.read_csv('Little_Red_Cap_ Jacob_and_Wilhelm_Grimm.txt', lineterminator = '\n', header=None, sep="\t", names=['origin_sentences'])
df.head(5)

,origin_sentences
0,Little Red Cap
1,Jacob and Wilhelm Grimm
2,Once upon a time there was a sweet little girl...
3,"One day her mother said to her, ""Come Little R..."
4,Little Red Cap promised to obey her mother. Th...


In [1]:
text = '''Once upon a time there was a sweet little girl. Everyone who saw her liked her, but most of all her grandmother, who did not know what to give the child next. Once she gave her a little cap made of red velvet. Because it suited her so well, and she wanted to wear it all the time, she came to be known as Little Red Cap.
One day her mother said to her, "Come Little Red Cap. Here is a piece of cake and a bottle of wine. Take them to your grandmother. She is sick and weak, and they will do her well. Mind your manners and give her my greetings. Behave yourself on the way, and do not leave the path, or you might fall down and break the glass, and then there will be nothing for your sick grandmother."

Little Red Cap promised to obey her mother. The grandmother lived out in the woods, a half hour from the village. When Little Red Cap entered the woods a wolf came up to her. She did not know what a wicked animal he was, and was not afraid of him.

"Good day to you, Little Red Cap."

"Thank you, wolf."

"Where are you going so early, Little Red Cap?"

"To grandmother's."

"And what are you carrying under your apron?"
'''

### NLTK

In [3]:
import nltk

In [10]:
# All words

bag_of_words = nltk.word_tokenize(text)
bag_of_words

['Once',
 'upon',
 'a',
 'time',
 'there',
 'was',
 'a',
 'sweet',
 'little',
 'girl',
 '.',
 'Everyone',
 'who',
 'saw',
 'her',
 'liked',
 'her',
 ',',
 'but',
 'most',
 'of',
 'all',
 'her',
 'grandmother',
 ',',
 'who',
 'did',
 'not',
 'know',
 'what',
 'to',
 'give',
 'the',
 'child',
 'next',
 '.',
 'Once',
 'she',
 'gave',
 'her',
 'a',
 'little',
 'cap',
 'made',
 'of',
 'red',
 'velvet',
 '.',
 'Because',
 'it',
 'suited',
 'her',
 'so',
 'well',
 ',',
 'and',
 'she',
 'wanted',
 'to',
 'wear',
 'it',
 'all',
 'the',
 'time',
 ',',
 'she',
 'came',
 'to',
 'be',
 'known',
 'as',
 'Little',
 'Red',
 'Cap',
 '.',
 'One',
 'day',
 'her',
 'mother',
 'said',
 'to',
 'her',
 ',',
 '``',
 'Come',
 'Little',
 'Red',
 'Cap',
 '.',
 'Here',
 'is',
 'a',
 'piece',
 'of',
 'cake',
 'and',
 'a',
 'bottle',
 'of',
 'wine',
 '.',
 'Take',
 'them',
 'to',
 'your',
 'grandmother',
 '.',
 'She',
 'is',
 'sick',
 'and',
 'weak',
 ',',
 'and',
 'they',
 'will',
 'do',
 'her',
 'well',
 '.',
 'M

In [5]:
tokens_sens = nltk.tokenize.sent_tokenize(text, language='english')

In [11]:
pd.set_option('display.max_colwidth', None)
df_sentences = pd.DataFrame({'sentence': tokens_sens})
df_sentences

,sentence
0,Once upon a time there was a sweet little girl.
1,"Everyone who saw her liked her, but most of all her grandmother, who did not know what to give the child next."
2,Once she gave her a little cap made of red velvet.
3,"Because it suited her so well, and she wanted to wear it all the time, she came to be known as Little Red Cap."
4,"One day her mother said to her, ""Come Little Red Cap."
5,Here is a piece of cake and a bottle of wine.
6,Take them to your grandmother.
7,"She is sick and weak, and they will do her well."
8,Mind your manners and give her my greetings.
9,"Behave yourself on the way, and do not leave the path, or you might fall down and break the glass, and then there will be nothing for your sick grandmother."""


In [ ]:
pd.set_option('display.max_colwidth', 50)

### Spacy

In [29]:
!pip install spacytextblob
!python -m textblob.download_corpora
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.5/636.5 kB 4.5 MB/s eta 0:00:0000:0100:01
[nltk_data] Downloading package brown to /Users/Sergey/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /Users/Sergey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Sergey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Sergey/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     /Users/Sergey/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/Sergey/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 5.8 MB/s eta 0:00:0000:0100:01
  Attempt

In [31]:
import spacy
#from spacy.lang.ru import Russian
#from spacy import displacy
#from spacy.lang.en.examples import sentences 

#import en_core_web_sm
#nlp = en_core_web_sm.load()

nlp = spacy.load("en_core_web_sm")
doc = nlp("New Apple MacBook set launch tomorrow")

In [32]:
displacy.render(doc, style='dep', jupyter=True)

In [33]:
token = doc[0]
print(token.text)

New


In [24]:
nlp = Russian()
doc = nlp("Съешь ещё этих мягких французских булок, да выпей чаю.")

In [25]:
token = doc[0]
print(token.text)

span = doc[3:6]
print(span.text)

Съешь
мягких французских булок


In [52]:
print("is_alpha:    ", [token.is_alpha for token in doc])
print("is_punct:    ", [token.is_punct for token in doc])
print("like_num:    ", [token.like_num for token in doc])

is_alpha:     [True, True, True, True, True, True, False, True, True, True, False]
is_punct:     [False, False, False, False, False, False, True, False, False, False, True]
like_num:     [False, False, False, False, False, False, False, False, False, False, False]
